In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool,OpenAIChatCompletionsModel, output_guardrail, input_guardrail, output_guardrail,InputGuardrailTripwireTriggered, GuardrailFunctionOutput,RunContextWrapper,TResponseInputItem, OutputGuardrailTripwireTriggered
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
from openai import  AsyncOpenAI
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio
from pydantic import BaseModel

In [269]:
load_dotenv(override=True)

True

In [ ]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("srinivasanms16@gmail.com")  # Change to your verified sender
    to_email = To("srinivasanms16@gmail.com")  # Change to your recipient
    content = Content("text/html", body)
    mail = Mail(from_email, to_email,"sub", content).get()
    sg.client.mail.send.post(request_body=mail)
    #return {"status": "success"}

In [271]:
client = AsyncOpenAI(base_url="https://openrouter.ai/api/v1",api_key=os.getenv("OPENROUTER_API_KEY"))
openRouterModel = OpenAIChatCompletionsModel(openai_client=client,model="tngtech/deepseek-r1t2-chimera:free")

In [ ]:
class MessageOutput(BaseModel): 
    response: str

class OutputValidation(BaseModel):
    reasoning:str
    is_complain : bool

responseValidator = Agent(name="Response_Validator",
                          instructions="Check if the email is complain",
                          model="gpt-4.1-mini",
                          output_type=OutputValidation)

@output_guardrail
async def validateResponse(
   ctx: RunContextWrapper[None], agent: Agent, input: str | list[TResponseInputItem]
    ) -> GuardrailFunctionOutput:
    print("Output Guardrail is called..!")
    response = await Runner.run(starting_agent=responseValidator,context=ctx.context, 
                                input=input)
    return GuardrailFunctionOutput(output_info=response.final_output, 
                                   tripwire_triggered=response.final_output.is_complain)
    

In [273]:
#Bussiness Email Generator
BussinessEmailGenerator = Agent(name="Bussiness_Email_Generator",
                                 instructions="Generate the email in bussiness or professional tone",
                                 model=openRouterModel)
#Bussiness Email Generator Tool
BussinessEmailGeneratorTool = BussinessEmailGenerator.as_tool(tool_name="Bussiness_Email_Generator_tool",
                                                              tool_description="Generate the email in bussiness or professional tone")
#Friendly Email Generator 
friendlyEmailGenerator = Agent(name="Friendly_Email_Generator",
                               instructions="Generator the email in friendly or unprofessional tone",
                               model=openRouterModel)
#Friendly Email Generator Tool
friendlyEmailGeneratorTool = friendlyEmailGenerator.as_tool(tool_name="friendly_Email_Generator_Tool",
                                                            tool_description="Generator the email in friendly or unprofessional tone")
# HTML Formator and email sender 
formaterandsender = Agent(name="formater_sender",
                 instructions="your a agent who will apply html format to the email and send the email",
                 tools=[send_email],
                 output_guardrails=[validateResponse])


#Email Generator Agent
EmailGeneratorSender = Agent(name="Email_generator",
                       instructions="Generator email depending upon the request and format the email and send the email",
                       model ="gpt-4.1-mini",
                       tools=[BussinessEmailGeneratorTool,friendlyEmailGeneratorTool],
                       handoffs=[formaterandsender])



In [274]:
class PersonalInfoPresent(BaseModel):
    input:str
    is_Personal_Info_avaliable:bool

InputGuardrail = Agent(name="PersonalInfoCheck",
                       instructions="Check is any personal information such as name or emailid is present"
                       ,model="gpt-4.1-mini",
                       output_type=PersonalInfoPresent)

In [275]:
@input_guardrail
async def checkPersonalInfoPresent( 
    ctx, agent, input
) -> GuardrailFunctionOutput:
    response = await Runner.run(starting_agent=InputGuardrail,input=input,context=ctx.context)
    return GuardrailFunctionOutput(output_info=response.final_output, 
                                   tripwire_triggered=response.final_output.is_Personal_Info_avaliable)


In [276]:
NotifierAgent = Agent(name="Notifier",
                      instructions="your the Notifier. who will have to Notify to user",
                      handoffs=[EmailGeneratorSender],
                      input_guardrails=[checkPersonalInfoPresent],
                      output_guardrails=[validateResponse],
                      model="gpt-4.1-mini",
                      output_type=MessageOutput 
                      )

print(NotifierAgent)


Agent(name='Notifier', instructions='your the Notifier. who will have to Notify to user', prompt=None, handoff_description=None, handoffs=[Agent(name='Email_generator', instructions='Generator email depending upon the request and format the email and send the email', prompt=None, handoff_description=None, handoffs=[Agent(name='formater_sender', instructions='your a agent who will apply html format to the email and send the email', prompt=None, handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, response_include=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[FunctionTool(name='send_email', description='Send out an email with the given body', params_json_schema={'properties': {'body': {'title': 'Body', 'type

In [278]:
with trace("Notifier"):
    try:
        result = await Runner.run(NotifierAgent,
                              "notify user using email regarding my Complain on customer service in 50 words and send it.")
        print(result.final_output)
    except InputGuardrailTripwireTriggered:
        print("Validation failed: you have provided the Personal Information")
    
    except OutputGuardrailTripwireTriggered:
         print("Validation failed: its has generated the complain email")

its called..!
The email regarding your complaint on customer service has been formatted and sent successfully. If there's anything else you need, feel free to ask!
